# Tarea cambio Portafolio VaR 3

> 4 de septiembre de 2020

1. Calcular el VaR y Expected Shortfall para el viernes 4 de septiembre de 2020 del siguiente portafolio, simulación histórica (2 años) nivel de confianza 97.5%

| Activo  | IYW | IBB  | IYF  | SPY | GLD  |  VXX  |
|---------|-----|------|------|-----|------|-------|
| Títulos | 644 | 1887 | 2130 | 286 | 540  | 3703  |

**a)** ¿Cuál habria sido la pérdida del portafolio si no hubieramos comprado ayer el VXX?

**b)** ¿De cuánto sería nuestro VaR y ES para mañana si no tuvieramos el VXX en el portafolio?

In [23]:
# Importar paqueterías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_datareader.data as web

In [24]:
# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

In [25]:
def  VaRPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                    start: 'Fecha inicio',
                    end: 'Fecha final',
                    titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                    confianza: 'Nivel de confianza del VaR (en decimal)'
                    ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Acomodarlo menor a mayor
    sortport= np.sort(Port)
    n= len(Port)
    # Nivel de confianza
    fuera= round(n*(1-confianza))
    # VaR en porcentaje
    VaRPortP= sortport[fuera]
    # VaR en dinero
    VaRPortM= ExpTotal*VaRPortP
    return print("VaR en porciento:", VaRPortP, 
                 '\nVaR en dinero:', VaRPortM)

In [26]:
def  ESPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                   start: 'Fecha inicio',
                   end: 'Fecha final',
                   titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                   confianza: 'Nivel de confianza del VaR (en decimal)'
                   ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Acomodarlo menor a mayor
    sortport= np.sort(Port)
    n= len(Port)
    # Nivel de confianza
    fuera= round(n*(1-confianza))
    # ES en porcentaje
    ESPortP= np.mean(sortport[0:fuera])
    # ES en dinero
    ESPortM= ESPortP*ExpTotal
    return print("ES en porciento:", ESPortP, 
                 '\nES en dinero:', ESPortM)

In [30]:
names=['IYW', 'IBB', 'IYF', 'SPY', 'GLD', 'VXX']
start= '2018-09-03'
end= '2020-09-03'
titulos= [644, 1887, 2130, 286, 540, 3703]
confianza= 0.975
VaRPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

VaR en porciento: -0.01754824506166703 
VaR en dinero: -17747.65387835992


In [31]:
ESPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

ES en porciento: -0.031446744696723754 
ES en dinero: -31804.088586484853


## a) ¿Cuál habria sido la pérdida del portafolio si no hubieramos comprado ayer el VXX?

In [45]:
def  RendPortafolio(names: 'Tickers de los activos del portafolio en []', 
                    start: 'Fecha inicio',
                    end: 'Fecha final',
                    titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                    ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start, end_date=end)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    ret['Port']= np.sum(PortIndiv, axis=0)
    return ret['Port']

In [46]:
names2= ['IYW', 'IBB', 'IYF', 'SPY', 'GLD']
titulos2= [644, 1887, 2130, 286, 540]
PortsVXX= RendPortafolio(names=names2, start=start, end=end, titulos=titulos2)

In [50]:
PortDin= PortsVXX[-1]*893941.34
print('De no haber agregado VXX al portafolio habríamos obtenido una pérdida porcentual de {}.'
      .format(PortsVXX[-1]))
print('Sabiendo que el Exposure total fue de $893941.34, habríamos tenido una pérdida en dinero de ${}.'
      .format(PortDin))

De no haber agregado VXX al portafolio habríamos obtenido una pérdida porcentual de -0.03373306564421819.
Sabiendo que el Exposure total fue de $893941.34, habríamos tenido una pérdida en dinero de $-30155.38190430037.


## b) ¿De cuánto sería nuestro VaR y ES para mañana si no tuvieramos el VXX en el portafolio?

In [32]:
VaRPortTitulos(names=names2, start=start, end=end, titulos=titulos2, confianza=confianza)

VaR en porciento: -0.030395430908644758 
VaR en dinero: -27171.732097564556


In [33]:
ESPortTitulos(names=names2, start=start, end=end, titulos=titulos2, confianza=confianza)

ES en porciento: -0.05228467185298456 
ES en dinero: -46739.4293789834


___
# Tarea cambio Portafolio VaR 2

> 3 de septiembre de 2020

1. Calcular el VaR y Expected Shortfall para el jueves 3 de septiembre de 2020 del siguiente portafolio, simulación histórica (2 años) nivel de confianza 97.5%

| Activo  | IYW | IBB  | IYF  | SPY | GLD  |  VXX  |
|---------|-----|------|------|-----|------|-------|
| Títulos | 644 | 1887 | 2130 | 286 | 540  | 3703  |

**a)** ¿Al agregr este activo, subió el VaR o bajó? ¿En cuánto?

In [18]:
names=['IYW', 'IBB', 'IYF', 'SPY', 'GLD', 'VXX']
start= '2018-09-02'
end= '2020-09-02'
titulos= [644, 1887, 2130, 286, 540, 3703]
confianza= 0.975
VaRPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

VaR en porciento: -0.0189768917465918 
VaR en dinero: -19512.49511072851


In [19]:
ESPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

ES en porciento: -0.033798933354820836 
ES en dinero: -34752.87368661019


## a) ¿Al agregr este activo, subió el VaR o bajó? ¿En cuánto?
> Cálculos arriba y abajo

- Como podemos observar, agregando VXX fuimos capaces de reducir el VaR del portafolio de -26082.44 usd a -19512.49 usd.
- El Expected Shortfall se redujo un total de 12843.93 usd.
- En ambos escenarios fue una buena idea desde el punto de vista de riesgo agregar este asset.


___
# Tarea cambio Portafolio VaR

> 2 de septiembre de 2020

1. Calcular el VaR y Expected Shortfall para el miércoles 2 de septiembre de 2020 del siguiente portafolio, simulación histórica (2 años) nivel de confianza 97.5%

| Activo  | IYW | IBB  | IYF  | SPY | GLD  |
|---------|-----|------|------|-----|------|
| Títulos | 644 | 1887 | 2130 | 286 | 540  |

In [21]:
names=['IYW', 'IBB', 'IYF', 'SPY', 'GLD']
start= '2018-09-01'
end= '2020-09-01'
titulos= [644, 1887, 2130, 286, 540]
confianza= 0.975
VaRPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

VaR en porciento: -0.028507945798715743 
VaR en dinero: -26082.44551228855


In [147]:
def  ESPortTitulos(names: 'Tickers de los activos del portafolio en []', 
                   start: 'Fecha inicio (se asume hoy fecha final)',
                   titulos: 'Titulos de los activos del portafolio en [] (mismo orden de tickers)',
                   confianza: 'Nivel de confianza del VaR (en decimal)'
                   ):
    # Descarga de datos
    closes= get_adj_closes(tickers=names, start_date=start)
    ret= np.log(closes/closes.shift(1)).iloc[1:]
    # Exposure 
    ExpTotal= np.sum([titulos[i]*closes[names[i]][-1] for i in range(len(names))])
    ExpIndiv= [titulos[j]*closes[names[j]][-1] for j in range(len(names))]
    # Pesos
    PesosAct= [ExpIndiv[k]/ExpTotal for k in range(len(ExpIndiv))]
    # Portafolio
    PortIndiv= [PesosAct[i]*ret[names[i]] for i in range(len(PesosAct))]
    Port= np.sum(PortIndiv, axis=0)
    # Acomodarlo menor a mayor
    sortport= np.sort(Port)
    n= len(Port)
    # Nivel de confianza
    fuera= round(n*(1-confianza))
    # ES en porcentaje
    ESPortP= np.mean(sortport[0:fuera])
    # ES en dinero
    ESPortM= ESPortP*ExpTotal
    return print("ES en porciento:", ESPortP, 
                 '\nES en dinero:', ESPortM)

In [22]:
ESPortTitulos(names=names, start=start, end=end, titulos=titulos, confianza=confianza)

ES en porciento: -0.05202300245680959 
ES en dinero: -47596.80464337476


___
# Tarea reducir VaR Portafolio

> 1 de septiembre de 2020

1. Calcular el VaR y Expected Shortfall para el martes 1 de septiembre de 2020 del siguiente portafolio, simulación histórica (2 años) nivel de confianza 97.5%

| Activo  | IYW | IBB  | IYF  | SPY | GLD  |
|---------|-----|------|------|-----|------|
| Títulos | 944 | 1887 | 1676 | 286 | 540  |

2. Si nos dan la orden de reducir el VaR a 20 mil dólares o menos,explica detalladamente cómo hacerlo con:

**a)** Futuros de algún indice bursátil 

**b)** Reduciendo en alguno de los activos actuales para comprar VXX o algún etf inverso de algún índice bursátil (del Dow, S&P o Nasdaq)

**c)** Con calls ya sea de los activos que tiene el portafolio o de algún índice bursátil 

**d)** Con puts ya sea de los activos que se traen en el portafolio o de algún índice

## 1.

In [115]:
names=['IYW', 'IBB', 'IYF', 'SPY', 'GLD']
start= '2018-08-31'
titulos= [944, 1887, 1676, 286, 540]
confianza= 0.975
VaRPortTitulos(names=names, start=start, titulos=titulos, confianza=0.975)

VaR en porciento: -0.029833930224819942 
VaR en dinero: -28522.417406775014


In [116]:
ESPortTitulos(names=names, start=start, titulos=titulos, confianza=0.975)

ES en porciento: -0.052246026335900544 
ES en dinero: -49947.539055113346


## 2.

### a) Futuros de algún indice bursátil
> Cálculos abajo.

- Nuestro VaR es de -28522.42 usd y queremos reducirlo a menos de 20 mil dólares, por lo que se realizará una cobertura por 9k.
- Se realizará la cobertura que con el índice S&P500 mini, que tiene un valor de 50 dls por punto.
- Como podemos observar, el activo SPY ya se encuentra en el portafolio, por lo que se puede realizar una cobertura directa.
- En todos los activos (incluido SPY) estoy largo, por lo que debo tomar una posición corta para poderme cubrir.
- Como no puede haber fracciones de contratos es necesario redondear, por lo que se determina que el número de contratos para la posición es **1**.

In [63]:
nombres= ['IYW', 'IBB', 'IYF', 'GLD', 'SPY']
closes= get_adj_closes(tickers=nombres, start_date='2020-07-31')
closes.head()

Symbols,GLD,IBB,IYF,IYW,SPY
Date,,,,,
2020-07-31,185.429993,134.369995,113.739998,285.959991,326.519989
2020-08-03,185.639999,138.570007,113.620003,292.299988,328.790009
2020-08-04,189.589996,137.679993,113.739998,293.059998,330.059998
2020-08-05,191.350006,137.279999,115.139999,293.440002,332.109985
2020-08-06,193.889999,136.759995,115.250000,297.820007,334.329987


In [117]:
# Tomando en cuenta que cada punto del S&P500 vale 50 usd haremos los sig. cálculos
# para determinar el núm de contratos necesarios
SPYhoy= 50*closes['SPY'][-1]
cContratos= 9000/SPYhoy
print('Contratos necesarios {}.'.format(cContratos),
      '\nRedondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.')

Contratos necesarios 0.5153015981737155. 
Redondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.


### b) Reduciendo en alguno de los activos actuales para comprar VXX o algún etf inverso de algún índice bursátil (del Dow, S&P o Nasdaq)
> Cálculos abajo.

- Para decidir cuál activo reducir observé su volatilidad para ver cuál era el más riesgoso.
- El asset más volatil fue el GLD, seguido por IBB, por lo que me dispuse a reducir el número de títulos y agregar de VXX.
- Después de probar con diferentes combinaciones decidí eliminar por completo GLD y reducir 300 títulos de IBB para agregar 3000 de VXX, así pude reducir el VaR a menos de 20k usd de manera satisfactoria.

In [119]:
# Rendimientos
ret= np.log(closes/closes.shift(1)).iloc[1:]
# Volatilidad
nombres= ['IYW', 'IBB', 'IYF', 'GLD', 'SPY']
sActivos= [ret[nombres[i]].std() for i in range(len(nombres))]
sActivos

[0.01125921774367927,
 0.012598114047430583,
 0.008052364958292611,
 0.01856666521239993,
 0.005143106611940606]

In [127]:
namen= ['IYW', 'IBB', 'IYF', 'SPY', 'VXX']
start= '2018-08-31'
titulos= [944, 1587, 1676, 286, 3000]
confianza= 0.975
VaRPortTitulos(names=namen, start=start, titulos=titulos, confianza=confianza)

VaR en porciento: -0.02197186224356331 
VaR en dinero: -19716.482404168848


### c) Con calls ya sea de los activos que tiene el portafolio o de algún índice bursátil
> Cálculos abajo.

- Nuestro VaR es de -28522.42 usd y queremos reducirlo a menos de 20 mil dólares, por lo que se realizará una cobertura por 9k.
- Como recordamos del inciso anterior, el asset con mayor volatilidad es el de GLD, por lo que buscaremos una opción del mismo.
- Elijo usar calls a 10 días con strike de 185 y delta de 0.5461.
- Entonces si cada contrato es de 100 títulos, implica que cada call equivale a 18500 usd.
- Pero dado que su delta es de 0.5 significa que en realidad se mueve como si se tratara de 50 títulos.
- Como no puede haber fracciones de contratos es necesario redondear, por lo que se determina que el número de contratos call para la posición es **1**.
- Entonces, **estoy corto en un call de GLD**.

In [128]:
# Vemos como cerró GLD
closes['GLD'][-1]

184.8300018310547

In [133]:
# A cuanto equivale cada call
Call= 185*0.5461*100
print('Cada call de los mencionados equivale a ${}.'.format(Call))

Cada call de los mencionados equivale a $10102.85.


In [135]:
# Calls a comprar
cCalls= 9000/Call
print('Contratos necesarios {}.'.format(cCalls),
      '\nRedondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.')

Contratos necesarios 0.8908377339067689. 
Redondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.


### d) Con puts ya sea de los activos que se traen en el portafolio o de algún índice
> Cálculos abajo.

- Nuestro VaR es de -28522.42 usd y queremos reducirlo a menos de 20 mil dólares, por lo que se realizará una cobertura por 9k.
- Como recordamos del inciso anterior, el asset con mayor volatilidad es el de GLD, por lo que buscaremos una opción del mismo.
- Elijo usar putss a 10 días con strike de 185 y delta de -0.4544.
- Entonces si cada contrato es de 100 títulos, implica que cada call equivale a 18500 usd.
- Pero dado que su delta es de 0.5 significa que en realidad se mueve como si se tratara de 50 títulos.
- Como no puede haber fracciones de contratos es necesario redondear, por lo que se determina que el número de contratos call para la posición es **1**.
- Entonces, **estoy corto en un put de GLD**.

In [140]:
# A cuanto equivale cada put
Put= 185*0.4544*100
print('Cada call de los mencionados equivale a ${}.'.format(Put))

Cada call de los mencionados equivale a $8406.400000000001.


In [141]:
# Puts a comprar
cPuts= 9000/Put
print('Contratos necesarios {}.'.format(cPuts),
      '\nRedondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.')

Contratos necesarios 1.070612866387514. 
Redondeamos hacia arriba por lo que nos quedamos con UN contrato para la cobertura.


___
# Tarea VaR Portafolio

> 31 de agosto de 2020

1. Calcular el VaR y Expected Shortfall para el lunes 31 de agosto de 2020 del siguiente portafolio. Simulación histórica (2 años). Nivel de confianza 97.5%.

| Activo  | IYW | IBB  | IYF  | SPY | SLV  |
|---------|-----|------|------|-----|------|
| Títulos | 944 | 1887 | 1676 | 286 | 7785 |

### VaR 

In [26]:
names=['IYW', 'IBB', 'IYF', 'SPY', 'SLV']
start= '2018-08-30'
titulos= [944, 1887, 1676, 286, 7785]
confianza= 0.975
VaRPortTitulos(names=names, start=start, titulos=titulos, confianza=0.975)

VaR en porciento: -0.03053655327075008 
VaR en dinero: -32371.453133052786


### ES

In [27]:
ESPortTitulos(names=names, start=start, titulos=titulos, confianza=0.975)

ES en porciento: -0.052657680242071604 
ES en dinero: -55821.80847123582


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Edgar Machuca Buenrostro.
</footer>